# Library

In [24]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import pprint
import time

# Database

In [25]:
client = MongoClient('203.255.92.141:27017', connect=False)
ID = client['ID']
ID_Domestic = ID['Domestic']
ntis_client  = client['NTIS']
ntis_rawdata = ntis_client['Rawdata']
scienceon = client['SCIENCEON']
scienceon_authorpapers = scienceon['AuthorPapers']
scienceon_rawdata = scienceon['Rawdata']

# Program

In [26]:
scienceon_id = []
papers_content = []
scienceon_dict = {}
ntis_data = []

In [27]:
#NTIS author ID and papers
AuthorPapers_A_ID_AND_Papers = list(ntis_client['AuthorPapers'].find({'keyId':519}, {'A_ID':1,"papers":1}))
for A_ID_Data in AuthorPapers_A_ID_AND_Papers[0:46]:
    scienceon_author = []
    #scienceon_data = []
    scienceon_dict['ntis'] = A_ID_Data['A_ID'] #NTIS ID
    paper_cnt = [] #Number of papers
    paper_cnt.append(len(A_ID_Data['papers']))
    print(paper_cnt)
    papers_content = []
    # keyid 519로 검색했을 경우의 논문
    for id_paper in A_ID_Data['papers']: #NTIS Papers content
        ntis_paper_data = list(ntis_rawdata.find({"_id":id_paper},{"koTitle":1,"enTitle":1,"absAbs":1,"effAbs":1,"koKeyword":1,"enKeyword":1,"goalAbs":1}))
        papers_content.append(ntis_paper_data[0])
    #print(scienceon_dict)
    
    # keyid 519로 검색했을 경우 나오는 a_id에 대한 scienceon의 id
    author = ID_Domestic.find({"ntis":A_ID_Data['A_ID']},{"scienceon":1,"ntis":1})
    for id in author:
        for scienceon_id in id["scienceon"]:
            #paper_cnt.append(len(A_ID_Data['papers']))
            scienceon_author.append(scienceon_id) #SCIENCEON ID
            #print(scienceon_author)
            a = scienceon_authorpapers.find({"A_ID":scienceon_id, "keyId":519},{"papers":1})
            if a:
                paper_cnt.append(0)
            else:
                for scienceon_paper_id in a: #Scienceon Papers content
                    print(scienceon_paper_id['papers'])
                    #print(len(scienceon_paper_id['papers']))
                    #print(scienceon_id)
                    paper_cnt.append(len(scienceon_paper_id['papers']))
                    scienceon_paper_data = list(scienceon_rawdata.find({"_id":scienceon_paper_id['papers'][0]},{"title":1,"abstract":1,"english_title":1,"qryKeyword":1}))
                    papers_content.append(scienceon_paper_data[0])
        #print(scienceon_author)     
    scienceon_dict['scienceon'] = scienceon_author
    scienceon_dict['papers_number'] = paper_cnt
    scienceon_dict['papers'] = papers_content
    print(scienceon_dict)    

[1]
{'ntis': '100123546003', 'scienceon': [], 'papers_number': [1], 'papers': [{'_id': ObjectId('615e76ea4c7d7dc530bde2a6'), 'koTitle': '공간지능을 위한 IoT 사물간 자율협업 기술 개발', 'enTitle': 'Development of Autonomous IoT Collaboration Framework for Space Intelligence', 'goalAbs': '... 최적화시키는 공간지능 자가 적응 핵심 기술 개발   - 정확한 공간 맥락 인지를 위한 온톨로지 기반 지능 사물 상황정보 의미 확장 및 협업적 상황인지 기술 (S/W) 개발   - 공간지능 빅데이터 기반 지속적 공간 지식 자가 진화 기술 (S/W) 개발   - 공간 속 사용자 의도 파악 및 추적을 위한 사용자 군집 기반 페르소나 생성 기술 (S/W) 개발   - 공간 속 사용자 의도와 ...', 'absAbs': '... 온톨로지 및 워드 임베딩 기반 지능 사물 정보의 의미 확장 기술 - 온톨로지 기반 지능 사물들의 데이터 정형화 및 지식 체계 구축 - Word Embedding을 이용한 지능 사물의 의미정보 확장 ... 컨텍스트 수집/분석 기반 페르소나 생성 기술 연구 - 사용자들이 자발적으로 참여하여 제공하는 데이터 (예: 소셜 미디어 사진) 를 이용한 탐색적 데이터 수집 및 분석 기술 - 대용량 데이터 기반의 ...', 'effAbs': '1차년도는 동적 IoT 공간에서 분산 지능 사물들이 공간의 변화 및 사용자의 의도를 파악하고 최적의 서비스를 제공하기 위해 자율적으로 협업 방법을 학습 및 최적화시키는 공간지능 자가 적응 핵심 기술의 기반을 이루는 모델들에 대한 설계 및 실증에 포커스를 두고 있다. ○ 1차년도의 결과물로 기대되는 본 과제의 핵심 모델:-  온톨로지 기반의 분산 지식 모델-...', 'koKeyword': '공간 지능,사물 인터넷,사용자 의도,자율 사물 협업

{'ntis': '111405177161', 'scienceon': [], 'papers_number': [1], 'papers': [{'_id': ObjectId('615e76f14c7d7dc530bde3b0'), 'koTitle': '물류 및 교통정보 스트리밍 빅 데이터 처리 기술 및 분산/병렬 플랫폼 기술 개발', 'enTitle': 'Distributed/Parallel Platform of Streaming Big Data Processing for Logistics and Transportation Applications', 'goalAbs': '* 연구센터의 설립목적  o 물류와 교통 정보 서비스의 빅 데이터 처리 문제를 중점적으로 연구할 특성화 연구센터로 육성지원  o 산업체에서 필요로 하는 빅 데이터 처리 기술을 보유하는 인재 배출을 위한 강한 특허 강한 특허: 법률적 관점에서 독점배타적으로 실시할 수 있는 권리를 강하게 내세울 수 있는 발명을 말함* 연구목표o 빅 데이터의 효율적인 저장관리, 분석, ...', 'absAbs': '[제1세부과제] 빅 데이터 분산/병렬 컴퓨팅 운용 및 통합 플랫폼 기술 연구 o 스트리밍 빅 데이터의 실시간 질의 처리 기술  - 실시간 스트림 데이터 처리를 위한 Map-Reduce ... 분산/병렬 프로그램 전용 Virtual Cluster Template 개발[제2세부과제] 빅 데이터 트렌드 분석 및 예측 기술 연구  o 빅데이터 트렌드 분석 연구  - Map-reduce ...', 'effAbs': 'o 산업적 효과  - 공공 부문인 물류 및 교통 분야에서 발생하는 빅 데이터의 처리/분석 및 플랫폼에 집중함으로써 국내 빅 데이터 기술의 국제적 경쟁력을 높이고, 국내외 시장의 상당 ... 다각적 분석 및 정책 시사점 도출  - Open API를 기반으로 다양한 모바일, 웹 및 소셜 네트워크 서비스와의 Mash-Up을 통한 지식 융합 서비스 제공 o 인력양성 효과  - 물류 교통 빅 ...', 'koKeyword'